<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 25s 963ms/step - loss: 0.6598 - accuracy: 0.6095 - val_loss: 0.6315 - val_accuracy: 0.6439
Epoch 2/10
26/26 [==============================] - 24s 904ms/step - loss: 0.6120 - accuracy: 0.6606 - val_loss: 0.6705 - val_accuracy: 0.5268
Epoch 3/10
26/26 [==============================] - 24s 915ms/step - loss: 0.5723 - accuracy: 0.7251 - val_loss: 0.7251 - val_accuracy: 0.5122
Epoch 4/10
26/26 [==============================] - 24s 912ms/step - loss: 0.5208 - accuracy: 0.7506 - val_loss: 0.4830 - val_accuracy: 0.8390
Epoch 5/10
26/26 [==============================] - 27s 1s/step - loss: 0.4482 - accuracy: 0.8005 - val_loss: 0.4033 - val_accuracy: 0.8439
Epoch 6/10
26/26 [==============================] - 25s 947ms/step - loss: 0.4104 - accuracy: 0.8224 - val_loss: 0.3696 - val_accuracy: 0.8293
Epoch 7/10
26/26 [==============================] - 24s 934ms/step - loss: 0.3390 - accuracy: 0.8662 - val_loss: 0.2910 - val_accuracy: 0.8732
Ep

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 27s - loss: 0.6450 - accuracy: 0.6290 - val_loss: 0.6606 - val_accuracy: 0.5366
Epoch 2/5

Epoch 00002: saving model to weights.02-0.56.h5
26/26 - 25s - loss: 0.5952 - accuracy: 0.6813 - val_loss: 0.5604 - val_accuracy: 0.8049
Epoch 3/5

Epoch 00003: saving model to weights.03-0.52.h5
26/26 - 26s - loss: 0.5476 - accuracy: 0.7348 - val_loss: 0.5230 - val_accuracy: 0.7512
Epoch 4/5

Epoch 00004: saving model to weights.04-0.44.h5
26/26 - 24s - loss: 0.4902 - accuracy: 0.7786 - val_loss: 0.4421 - val_accuracy: 0.7951
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 24s - loss: 0.4115 - accuracy: 0.8248 - val_loss: 0.4076 - val_accuracy: 0.8683


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 26s - loss: 0.6792 - accuracy: 0.5900 - val_loss: 0.6738 - val_accuracy: 0.5220


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 22s - loss: 0.6746 - accuracy: 0.5681 - val_loss: 0.6717 - val_accuracy: 0.4829
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6430 - accuracy: 0.6910 - val_loss: 0.6242 - val_accuracy: 0.7951


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6583 - accuracy: 0.6071 - val_loss: 0.6288 - val_accuracy: 0.6927
Epoch 2/50
26/26 - 21s - loss: 0.6200 - accuracy: 0.6752 - val_loss: 0.5802 - val_accuracy: 0.6634
Epoch 3/50
26/26 - 21s - loss: 0.5672 - accuracy: 0.7238 - val_loss: 0.5143 - val_accuracy: 0.7707
Epoch 4/50
26/26 - 22s - loss: 0.4949 - accuracy: 0.7859 - val_loss: 0.6720 - val_accuracy: 0.6000
Epoch 5/50
26/26 - 21s - loss: 0.4475 - accuracy: 0.7981 - val_loss: 0.5399 - val_accuracy: 0.7122
Epoch 6/50
26/26 - 21s - loss: 0.3815 - accuracy: 0.8552 - val_loss: 0.3460 - val_accuracy: 0.8976
Epoch 7/50
26/26 - 21s - loss: 0.3696 - accuracy: 0.8516 - val_loss: 0.2782 - val_accuracy: 0.9073
Epoch 8/50
26/26 - 21s - loss: 0.2712 - accuracy: 0.9027 - val_loss: 0.2134 - val_accuracy: 0.9512
Epoch 9/50
26/26 - 21s - loss: 0.2202 - accuracy: 0.9319 - val_loss: 0.2789 - val_accuracy: 0.8927
Epoch 10/50
26/26 - 21s - loss: 0.1743 - accuracy: 0.9513 - val_loss: 0.1297 - val_accuracy: 0.9756
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 821ms/step - loss: 0.6728 - accuracy: 0.5730 - val_loss: 0.6557 - val_accuracy: 0.6341
Epoch 2/5
26/26 [==============================] - 21s 809ms/step - loss: 0.6261 - accuracy: 0.6825 - val_loss: 0.5940 - val_accuracy: 0.7951
Epoch 3/5
26/26 [==============================] - 21s 807ms/step - loss: 0.5660 - accuracy: 0.7555 - val_loss: 0.6427 - val_accuracy: 0.5463
Epoch 4/5
26/26 [==============================] - 21s 796ms/step - loss: 0.4796 - accuracy: 0.8041 - val_loss: 0.4260 - val_accuracy: 0.8341
Epoch 5/5
26/26 [==============================] - 21s 800ms/step - loss: 0.4073 - accuracy: 0.8431 - val_loss: 0.8892 - val_accuracy: 0.4927


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.572993,0.673125,0.634146,0.655712
1,1,0.682482,0.626371,0.795122,0.593967
2,2,0.755474,0.565982,0.546342,0.642714
3,3,0.804136,0.480226,0.834146,0.426035
4,4,0.843066,0.405069,0.492683,0.889206


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6784 - accuracy: 0.5535 - val_loss: 0.6792 - val_accuracy: 0.4780

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 806ms/step - loss: 0.6610 - accuracy: 0.5827 - val_loss: 0.6653 - val_accuracy: 0.6341

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 22s 839ms/step - loss: 0.6515 - accuracy: 0.6642 - val_loss: 0.6657 - val_accuracy: 0.5512

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 804ms/step - loss: 0.6463 - accuracy: 0.6423 - val_loss: 0.6625 - val_accuracy: 0.5512

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 811ms/step - loss: 0.6429

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 859ms/step - loss: 0.6625 - accuracy: 0.6168 - val_loss: 0.6421 - val_accuracy: 0.6098
Epoch 2/50
26/26 [==============================] - 21s 811ms/step - loss: 0.6233 - accuracy: 0.6569 - val_loss: 0.5805 - val_accuracy: 0.7463
Epoch 3/50
26/26 [==============================] - 21s 827ms/step - loss: 0.5777 - accuracy: 0.7117 - val_loss: 0.5610 - val_accuracy: 0.6927
Epoch 4/50
26/26 [==============================] - 22s 850ms/step - loss: 0.5302 - accuracy: 0.7397 - val_loss: 0.4880 - val_accuracy: 0.7659
Epoch 5/50
26/26 [==============================] - 21s 811ms/step - loss: 0.4998 - accuracy: 0.7579 - val_loss: 0.4251 - val_accuracy: 0.8829
Epoch 6/50
26/26 [==============================] - 21s 816ms/step - loss: 0.4171 - accuracy: 0.8236 - val_loss: 0.3376 - val_accuracy: 0.9317
Epoch 7/50
26/26 [==============================] - 21s 811ms/step - loss: 0.3940 - accuracy: 0.8431 - val_loss: 0.3028 - val_accuracy: 0.9073

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4252), started 0:17:41 ago. (Use '!kill 4252' to kill it.)